In [1]:
import numpy as np
from string import punctuation
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
from natasha import DatesExtractor, MoneyExtractor
from extractor import NumberExtractor
import datetime, re

In [2]:
tokenizer = RegexpTokenizer(r"[а-яёa-z0-9]+")

stemmer_mystem = Mystem()
datesextractor = DatesExtractor()
moneyextractor = MoneyExtractor()
numberextractor = NumberExtractor()

In [7]:
# #Пример входного словаря d:

# d = dict({'text':'Хочу узнать статус черной карты с котиком',
#          'Evolution':['01.01.2021'],
#          'Premium':['01.01.2021'],
#          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
#          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл']})  

# #Запуск:
# trigger(d)   

# #Возвращает тот же словарь с доп полем target:
# {'text': 'Хочу узнать статус черной карты с котиком',
#  'Evolution': ['01.01.2021'],
#  'Premium': ['01.01.2021'],
#  'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
#  'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
#  'Ипотека на Дом и земельный участок': ['5000000 долл',
#   '180 мес',
#   '10000 долл'],
#  'target': 'Evolution'}


# #Пример словаря с полным перечнем продуктов
# d = dict({'text':'Хочу узнать статус черной карты с котиком',
#          'Aurum':['01.01.2021'],
#          'Generation':['01.01.2021'],
#          'Evolution':['01.01.2021'],
#          'Premium':['01.01.2021'],
#          'Mir':['01.01.2021'],
#          'Emotion':['01.01.2021'],
#          'Кредит наличными':['5000000 долл', '180 мес', '10000 долл'],
#          'Кредит под залог недвижимости':['5000000 евро', '180 мес', '10000 евро'],
#          'Кредит (Рефинансирование)':['5000000 руб', '180 мес', '10000 руб'],
#          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
#          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Новостройки с господдержкой':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Новостройки':['5000000 руб', '180 мес', '10000 руб'],
#          'Рефинансирование ипотеки':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека Господдержка для семей с детьми':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Коммерческую недвижимость':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Дом и земельный участок':['5000000 руб', '180 мес', '10000 руб']})

#Формат в точности как в примере: 
# даты через точки, 
# сроки в месяцах (через "мес"), 
# валюты можно как в кратком (р, руб, долл), так и в полном формате (рублей, долларов)

In [5]:
def trigger(d):

    def return_results_cards(d,lst_cards,lst_indexes,product_cards,specify_card):
        #сканируем в тексте срок карты
        dats = NER_filter_cards(d['text'])
        print('NER recognized:',dats)
        if dats:#если указан срок карты в тексте
            #сверяем со сроками всех карт клиента
            par2=[]
            for j,i in enumerate(d.values()):#сканируем значения словаря
                matches = datesextractor(i[0])
                par1=[]
                for match in matches:
                    par1.append([match.fact.as_json['year'],match.fact.as_json['month']])  
                if par1:#если удалось распознать даты карт клиента (если они есть)
                    if len(par1[0]) == len(dats):#если есть совпадение по срокам
                        if par1[0] == dats:
                            par2.append(j)
                    else:
                        if par1[0][0] == dats[0]:
                            par2.append(j)
            
            if par2:
                if len(par2)==1:#если есть одно совпадение
                    d['target']=list(d.keys())[par2[0]] #нашлась карта с таким же сроком и она одна - выдаем ее в таргет
                    return d
                else:#если совпадений несколько
                    d['target']=specify_card#запрашиваем доп инфу
                    return d
            else:#если не удалось
                d['target']=specify_card#запрашиваем доп инфу
                return d

        #если срок карты не указан (или не удалось его отсканировать) - ищем по описаниям            
        dd=[]
        if len(lst_indexes)==1:#если нашлась одна карта по описанию
            card_name = product_cards[lst_indexes[0]]#определяем ее название
            if card_name in d:#если такая карта есть у клиента
                d['target'] = card_name #выдаем в таргет эту карту
                return d
            else:#если такой карты нет у клиента
                d['target']=specify_card #запрашиваем доп инфу
                return d 

        elif len(lst_indexes)>1: #если нашлось несколько карт по описанию
            lsss = [len(i) for i in lst_cards] #определяем число ключевых слов для каждой карты
            card_indexes = [i for i, j in enumerate(lsss) if j == max(lsss)] #определяем карту с макс числом ключевых слов
            if len(card_indexes)==1:#если макс только у одной карты

                card_name = product_cards[card_indexes[0]]#определяем ее название
                if card_name in d:#если такая карта есть у клиента
                    d['target'] = card_name#выдаем в таргет эту карту
                    return d 
                else:#если такой карты нет у клиента
                    d['target']=specify_card#запрашиваем доп инфу
                return d
            elif len(card_indexes)>1:#если макс у нескольких карт
                whc =[j for j in [product_cards[i] for i in card_indexes] if j in d]#определяем какая из карт есть у клиента
                if len(whc)==1:#если у клиента есть одна такая карта
                    d['target'] = whc#выдаем в таргет эту карту
                    return d 
                else:#если у клиента несколько таких карт или таких карт нет
                    d['target']=specify_card#запрашиваем доп инфу
                    return d
        else:
            return dd    
                
    def return_results_credits(d,lst_cards,lst_indexes,product_cards,specify_card):  

        #сканируем в тексте суммы и сроки кредитов
        dats = NER_filter_credits(d['text'])
        print('NER recognized:',dats)
        if any(dats):#если указан срок кредита или суммы
            #поиск кредитов клиента
            par1=[]
            par1_names=[]
            for j,i in enumerate(d.values()):#сканируем значения словаря
                if list(d.keys())[j] in product_cards:#если найденный продукт в списке продуктов
                    par2=[]
                    for ii in i:
                        #сканируем суммы
                        matches = moneyextractor(ii)
                        for match in matches:
                            par2.append([match.fact.as_json['integer'],match.fact.as_json['currency']]) 
                        #сканируем сроки 
                        textn = filter_text(ii)
                        lsst = textn.split()
                        ind_year = [ij for ij,s in enumerate(lsst) if s.isdigit()]
                        yearn=[]
                        for ij in ind_year:
                            try:
                                if lsst[ij+1]=='мес':
                                    if len(lsst[ij])<=3:
                                        yearn = int(lsst[ij])
                                        par2.append(yearn)
                                        par1_names.append(list(d.keys())[j])
                                        break
                            except:
                                1        
                    par1.append(par2)

            par1 = list(filter(None, par1)) #итоговый список кредитов клиента
            #par1  =[[[200000, 'RUB'], 150, [10000, 'RUB']], [[200000, 'RUB'], 50, [10000, 'RUB']]]    
            print('Client products in dict recognized:',list(zip(par1,par1_names)))
            if any(par1):#если у клиента есть кредиты
                
                par2=[]#сопоставляем с кредитами, найденными в тексте
                for w,k in enumerate(par1):
                    par3=[]
                    for i in k:
                        for j in dats:
                            try:
                                j=j[0]
                                if j==i:
                                    par3.append(w)
                            except:
                                if j==i:
                                    par3.append(w)
                    par2.append(len(par3))#сколько совпадений по каждому кредиту
                #par2 = [3, 2]
                print('Lengths of common parameters:',list(zip(par2,par1_names)))
                inss = [i for i, j in enumerate(par2) if j == max(par2)] #определяем кредит с макс числом совпадений
                #inss = [0]

                if len(inss)==1:
                    d['target'] = par1_names[inss[0]]#выдаем в таргет этот кредит
                    return d 
                elif len(inss)>1:#если макс у нескольких кредитов
                    d['target']=specify_card#запрашиваем доп инфу
                    return d 

            else:#если не удалось
                d['target']=specify_all#запрашиваем доп инфу
                return d
        
        #если срок карты не указан (или не удалось его отсканировать) - ищем по описаниям            
        dd=[]
        if len(lst_indexes)==1:#если нашлась одна карта по описанию
            card_name = product_cards[lst_indexes[0]]#определяем ее название
            if card_name in d:#если такая карта есть у клиента
                d['target'] = card_name #выдаем в таргет эту карту
                return d
            else:#если такой карты нет у клиента
                d['target']=specify_card #запрашиваем доп инфу
                return d 

        elif len(lst_indexes)>1: #если нашлось несколько карт по описанию
            lsss = [len(i) for i in lst_cards] #определяем число ключевых слов для каждой карты

            card_indexes = [i for i, j in enumerate(lsss) if j == max(lsss)] #определяем карту с макс числом ключевых слов
            if len(card_indexes)==1:#если макс только у одной карты

                card_name = product_cards[card_indexes[0]]#определяем ее название
                if card_name in d:#если такая карта есть у клиента
                    d['target'] = card_name#выдаем в таргет эту карту
                    return d 
                else:#если такой карты нет у клиента
                    d['target']=specify_card#запрашиваем доп инфу
                return d
            elif len(card_indexes)>1:#если макс у нескольких карт
                whc =[j for j in [product_cards[i] for i in card_indexes] if j in d]#определяем какая из карт есть у клиента
                if len(whc)==1:#если у клиента есть одна такая карта
                    d['target'] = whc#выдаем в таргет эту карту
                    return d 
                else:#если у клиента несколько таких карт или таких карт нет
                    d['target']=specify_card#запрашиваем доп инфу
                    return d
        else:
            return dd

    def filter_text(text, tokenizer=tokenizer, stemmer_mystem=stemmer_mystem):
        text = str(text)
        tokens = tokenizer.tokenize(text.lower())
        tokens = stemmer_mystem.lemmatize(" ".join(tokens))
        tokens = [token for token in tokens if token != " " 
                and token.strip() not in punctuation]
        return " ".join(tokens)    



    def NER_filter_cards(text,datesextractor=datesextractor):

        #             не распознает:
        #              - февраль 22
        #              - 03 22
        #             распознает:
        #              - февраль 2022
        #              - в 2022 году 
        #              - в 2022, в феврале
        #              - в 22 году, в феврале
        #              - в конце года (в этом году, текущего года, в нынешнем году), в феврале
        #              - в следующем году (будущего года), в феврале
        #              - в том году (в прошлом году), в феврале

        matches = datesextractor(text)
        par1=[]

        textn = filter_text(text)    
        months = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь',
            'ноябрь','декабрь']
        month_ind = [i+1 for i,j in enumerate(months) if j in textn]

        for match in matches:
            try: 
                #случай "в феврале 2022"
                par1.append([match.fact.as_json['year'],match.fact.as_json['month']])  
            except:
                #случай "в 2022 году в феврале"
                if month_ind:
                    par1.append([match.fact.as_json['year'],month_ind[0]])
                else:
                    par1.append([match.fact.as_json['year']]) 
            return par1[0] 

        if not par1:
            #проверяем случай "в этом году"
            if 'этот год' in textn or 'текущий год' in textn or 'нынешний год' in textn or 'в конец год' in textn:
                if month_ind:
                    par1.append([datetime.datetime.now().year,month_ind[0]])
                else:
                    par1.append([datetime.datetime.now().year]) 
                return par1[0]

            #проверяем случай "в следующем году"
            if 'следующий год' in textn or 'будущий год' in textn:
                if month_ind:
                    par1.append([datetime.datetime.now().year+1,month_ind[0]])
                else:
                    par1.append([datetime.datetime.now().year+1]) 
                return par1[0]

            #проверяем случай "в прошлом году"
            if 'прошлый год' in textn or 'тот год' in textn:
                if month_ind:
                    par1.append([datetime.datetime.now().year-1,month_ind[0]])
                else:
                    par1.append([datetime.datetime.now().year-1]) 
                return par1[0]

            #проверяем случай "в 2022"
            if re.search('[1-2][0-9]{3}', textn):
                yearn = int(re.search('[1-2][0-9]{3}', text)[0])
                if month_ind:
                    par1.append([yearn,month_ind[0]])
                else:
                    par1.append([yearn]) 
                return par1[0]

            #проверяем случай "в 22 году"
            lsst = textn.split()
            ind_year = [i for i,s in enumerate(lsst) if s.isdigit()]
            yearn=[]
            for i in ind_year:
                try:
                    if lsst[i+1]=='год':
                        yearn = int('20'+lsst[i])
                        break
                except:
                    1
            if yearn:
                if month_ind:
                    par1.append([yearn,month_ind[0]])
                else:
                    par1.append([yearn]) 
                return par1[0]

        return par1 
    
    def NER_filter_credits(text,moneyextractor=moneyextractor):
        
        #             не распознает:
        #              - кредит на 5000
        #             распознает:
        #              - кредит на 5000 рублей
        #              - кредит на сумму 5000
        #              - сроком на 15 лет (3 года)
        #              - сроком на 180 месяцев
        
        textn = filter_text(text)
        lsst = textn.split()

        #поиск сроков кредитов
        ind_year = [i for i,s in enumerate(lsst) if s.isdigit()]
        yearn=[]
        for i in ind_year:
            try:
                if lsst[i+1]=='год':#проверка случая на 15 лет
                    if len(lsst[i])<=2:#исключаем случай до 2015 года
                        yearn = int(lsst[i])*12
                        break
                elif lsst[i+1]=='месяц':#проверка случая на 180 месяцев
                    if len(lsst[i])<=3:#исключаем случай до 2015 года
                        yearn = int(lsst[i])
                        break
            except:
                1

        #поиск сумм кредитов их валюты - "кредит на сумму 5000 рублей"
        matches = moneyextractor(text)
        sum1=[]
        for match in matches:
            sum1.append([match.fact.as_json['integer'],match.fact.as_json['currency']])
        #если не нашли валюту - "кредит на сумму 5000"
        if not sum1:
            ind_year = [i for i,s in enumerate(lsst) if s.isdigit()]
            for i in ind_year:
                try:
                    if lsst[i-1]=='сумма':
                        sum1 = [[int(lsst[i]),'RUB']]
                        break
                except:
                    1
        par1=[yearn,sum1] 
        return par1

    #Вопросы для уточнения:
    specify_card = 'Уточните, по какой карте Вы хотите получить информацию?'
    specify_credit = 'Уточните, по какому кредиту Вы хотите получить информацию?'
    specify_ipoteka = 'Уточните, по какому ипотечному кредиту Вы хотите получить информацию?'
    specify_deposit = 'Уточните, по какому вкладу Вы хотите получить информацию?'
    specify_all = 'Уточните, по какому банковскому продукту Вы хотите получить информацию?'
    
    if 'text' in d:
        text = filter_text(d['text'])
        
        # ключевые описания продуктов:
        # по картам:
        product_cards = ['Aurum',
                         'Generation',
                         'Evolution',
                         'Premium',
                         'Mir',
                         'Emotion']
        card_aurum = ['карта','золотистый', 'черный','мастеркард','мастер']
        card_gener = ['карта','вертикальный','фиолетовый','мастеркард','мастер']
        card_evol = ['карта','черный','леопард','тигр','гепард','кот','котик','животное','виза','виса','мастеркард','мастер']
        card_prem = ['карта','черный','бесконечность','восьмерка','виза','виса','мастеркард','мастер']
        card_mir = ['карта','синий']
        card_emot = ['карта','кредитка','кредитный','белый','леопард','тигр','гепард','кот','котик','животное','виза','виса','мастеркард','мастер'] 


        # по кредитам:    
        product_credits = ['Кредит наличными',
                           'Кредит под залог недвижимости',
                           'Кредит (Рефинансирование)',
                           'Кредит на личное подсобное хозяйство']
        credit_cash = ['кредит']
        credit_estate = ['кредит']
        credit_refin = ['кредит']
        credit_farm = ['кредит']


        # по ипотеке:
        product_ipoteka = ['Ипотека на Вторичное жилье',
                       'Ипотека на Новостройки с господдержкой',
                       'Ипотека на Новостройки',
                       'Рефинансирование ипотеки',
                       'Ипотека Господдержка для семей с детьми',
                       'Ипотека на Коммерческую недвижимость',
                       'Ипотека на Дом и земельный участок']
        ipoteka_old = ['ипотека']
        ipoteka_state_new = ['ипотека']
        ipoteka_new = ['ипотека']
        ipoteka_refin = ['ипотека']
        ipoteka_state_family = ['ипотека']
        ipoteka_commer = ['ипотека']
        ipoteka_house_land = ['ипотека']

        # по вкладам:
        product_deposit = ['Выгодный','Выгоднее']
        deposit_1 = ['вклад', 'депозит']
        deposit_2 = ['вклад', 'депозит']

        # проверяем на наличие прямого названия продукта (и если данная карта действительно есть у клиента)
        # по картам:
        if [i for i in ['aurum','gold','золото','голд', 'голдовый','аурум','золотой'] if i in text.split()]:
            if 'Aurum' in d:
                d['target'] = 'Aurum'
                return d
            else:
                d['target'] = specify_card
                return d  

        if [i for i in ['generation','дженерейшн','генерейшн','генерейшнл','генерация'] if i in text.split()]:
            if 'Generation' in d:
                d['target'] = 'Generation'
                return d
            else:
                d['target'] = specify_card
                return d  

        if [i for i in ['evolution','эволюшн','эвалюшн','эволюция'] if i in text.split()]:
            if 'Evolution' in d:
                d['target'] = 'Evolution'
                return d
            else:
                d['target'] = specify_card
                return d  

        if [i for i in ['premium','премиум'] if i in text.split()]:
            if 'Premium' in d:
                d['target'] = 'Premium'
                return d
            else:
                d['target'] = specify_card
                return d  

        if [i for i in ['мир','долголетие'] if i in text.split()]:
            if 'Mir' in d:
                d['target'] = 'Mir'
                return d
            else:
                d['target'] = specify_card
                return d  

        if [i for i in ['emotion','эмоушн'] if i in text.split()]:
            if 'Emotion' in d:
                d['target'] = 'Emotion'
                return d
            else:
                d['target'] = specify_card
                return d  

        # по кредитам:
        if [i for i in ['кредит наличный','кредит наличные'] if i in text]:
            if 'Кредит наличными' in d:
                d['target'] = 'Кредит наличными'
                return d
            else:
                d['target'] = specify_credit
                return d  

        if [i for i in ['кредит под залог недвижимость','кредит под недвижимость'] if i in text]:
            if 'Кредит под залог недвижимости' in d:
                d['target'] = 'Кредит под залог недвижимости'
                return d
            else:
                d['target'] = specify_credit
                return d  

        if [i for i in ['кредит на личный подсобный хозяйство','кредит на личный хозяйство','кредит на хозяйство'] if i in text]:
            if 'Кредит на личное подсобное хозяйство' in d:
                d['target'] = 'Кредит на личное подсобное хозяйство'
                return d
            else:
                d['target'] = specify_credit
                return d  

        # по ипотеке:
        if [i for i in ['ипотека на вторичный жилье','вторичный жилье'] if i in text]:
            if 'Ипотека на Вторичное жилье' in d:
                d['target'] = 'Ипотека на Вторичное жилье'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['ипотека на новостройка с господдержка','новостройка с господдержка'] if i in text]:
            if 'Ипотека на Новостройки с господдержкой' in d:
                d['target'] = 'Ипотека на Новостройки с господдержкой'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['ипотека на новостройка'] if i in text]:
            if 'Ипотека на Новостройки' in d:
                d['target'] = 'Ипотека на Новостройки'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['рефинансирование ипотека'] if i in text]:
            if 'Рефинансирование ипотеки' in d:
                d['target'] = 'Рефинансирование ипотеки'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['ипотека господдержка для семья с ребенок'] if i in text]:
            if 'Ипотека Господдержка для семей с детьми' in d:
                d['target'] = 'Ипотека Господдержка для семей с детьми'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['ипотека на коммерческий недвижимость'] if i in text]:
            if 'Ипотека на Коммерческую недвижимость' in d:
                d['target'] = 'Ипотека на Коммерческую недвижимость'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['ипотека на дом и земельный участок','ипотека на дом','ипотека на земельный участок'] if i in text]:
            if 'Ипотека на Дом и земельный участок' in d:
                d['target'] = 'Ипотека на Дом и земельный участок'
                return d    
            else:
                d['target'] = specify_ipoteka
                return d
        
        # по вкладам:
        if [i for i in ['вклад выгодный','выгодный'] if i in text]:
            if 'Вклад выгодный' in d:
                d['target'] = 'Вклад выгодный'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  

        if [i for i in ['вклад выгоднее','выгоднее'] if i in text]:
            if 'Вклад выгоднее' in d:
                d['target'] = 'Вклад выгоднее'
                return d
            else:
                d['target'] = specify_ipoteka
                return d  
        
        # проверяем на наличие ключевых слов
        # по картам:
        result_aurum = [i for i in card_aurum if i in text.split()]
        result_gener = [i for i in card_gener if i in text.split()]
        result_evol = [i for i in card_evol if i in text.split()]
        result_prem = [i for i in card_prem if i in text.split()]
        result_mir = [i for i in card_mir if i in text.split()]
        result_emot = [i for i in card_emot if i in text.split()]

        lst_cards = [result_aurum,result_gener,result_evol,result_prem,result_mir,result_emot]
        lst_indexes = [i for i, j in enumerate(lst_cards) if j]
        if lst_indexes:
            dd = return_results_cards(d,lst_cards,lst_indexes,product_cards,specify_card)
            if dd:
                return dd

        # по кредитам:
        result_cash = [i for i in credit_cash if i in text.split()]
        result_estate = [i for i in credit_estate if i in text.split()]
        result_refin = [i for i in credit_refin if i in text.split()]
        result_farm = [i for i in credit_farm if i in text.split()]

        lst_credits = [result_cash,result_estate,result_refin,result_farm]
        lst_cred_indexes = [i for i, j in enumerate(lst_credits) if j]
        if lst_cred_indexes:
            dd = return_results_credits(d,lst_credits,lst_cred_indexes,product_credits,specify_credit)
            if dd:
                return dd    

        # по ипотеке:
        result_old = [i for i in ipoteka_old if i in text.split()]
        result_state_new = [i for i in ipoteka_state_new if i in text.split()]
        result_new = [i for i in ipoteka_new if i in text.split()]
        result_refin = [i for i in ipoteka_refin if i in text.split()]
        result_state_family = [i for i in ipoteka_state_family if i in text.split()]
        result_commer = [i for i in ipoteka_commer if i in text.split()]
        result_house_land = [i for i in ipoteka_house_land if i in text.split()]
        
        lst_ipot = [result_old,result_state_new,result_new,result_refin,result_state_family,result_commer,result_house_land]
        lst_ipot_indexes = [i for i, j in enumerate(lst_ipot) if j]

        if lst_ipot_indexes:
            dd = return_results_credits(d,lst_ipot,lst_ipot_indexes,product_ipoteka,specify_ipoteka)
            if dd:
                return dd 
            
        # по вкладам:
        result_deposit1 = [i for i in deposit_1 if i in text.split()]
        result_deposit2 = [i for i in deposit_2 if i in text.split()]
        
        lst_deposit = [result_deposit1, result_deposit2]
        lst_deposit_indexes = [i for i, j in enumerate(lst_deposit) if j]
        
        if lst_deposit:
            dd = return_results_deposits(d, lst_deposit, lst_deposit_indexes, product_deposit, specify_deposit)
            if dd:
                return dd

        d['target']=specify_all
        return d
    else:
        return []

Осталось сделать:
1. Дописать return_results_credits или сделать новую функцию
2. Посмотреть названия вкладов и адаптировать под это код

Сделать это за сегодня, 9-ое августа